# Домашнее задание №5

Асланов А.Б., ИУ9-11М

- Взять текст учебника и извлечь конструкции прилагательное-существительное
- Упорядочить двумя способами
  - по частотности
  - по мере взаимной информации

- Оценить, при каком методе терминов больше среди первых 20 словосочетаний

В отчете д.б. не только показано, что именно извлечено, но и проведен анализ, почему словосочетание  - термин или не термин

In [12]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import collections
import pandas as pd
from pymystem3 import Mystem
import json
from itertools import tee

In [13]:
with open('Введение_в_юриспруденцию.txt', 'r') as f:
    text = f.read()

In [14]:
# Обработка для получения лемм
lemm_text = ''.join(m.lemmatize(text))

In [15]:
# Удаляем знаки препинания
lemm_text = re.split("[\n;–,'''` .?!:'«»()]", lemm_text)

# Удаляем пустые строки
lemm_text = list(filter(None, lemm_text)) #None - потому что фильтрует пустые строки
words = [word.lower() for word in lemm_text] # все слова приводим к нижнему регистру

In [16]:
# Каждому слову присваиваем свою часть речи
m = Mystem()

init_text = ''
for word in words:
    init_text += str(word.lower())
    init_text += ' '

ws = m.analyze(init_text)
tags = []
for w in ws:
    try:
        word = w['text']
        tag = w['analysis'][0]['gr'].split(',')[0].split('=')[0]
        tags.append([word, tag])
    except:
        pass

tags = dict(tags)

In [17]:
# Строим словосочетания, где первым словом является прилагательное, а вторым - существительное
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

adj_noun_bigrams = [k for k, v in zip(pairwise(tags.keys()), pairwise(tags.values())) if v[0] == 'A' and v[1] == 'S']
print(adj_noun_bigrams[:5000])

[('социальный', 'направленность'), ('юридический', 'профессия'), ('профессиональный', 'группа'), ('генеральный', 'сообщество'), ('правовой', 'регулирование'), ('основной', 'направление'), ('российский', 'федерация'), ('законотворческий', 'процесс'), ('политический', 'отношение'), ('противоречивый', 'характер'), ('правотворческий', 'задача'), ('господствующий', 'класс'), ('действующий', 'законодательство'), ('правильный', 'мотивация'), ('точный', 'сожаление'), ('неотложный', 'решение'), ('неопределенный', 'время'), ('федеральный', 'декабрь'), ('физический', 'лицо'), ('обозримый', 'перспектива'), ('активный', 'участие'), ('разнообразный', 'форма'), ('первоначальный', 'рабочий'), ('последний', 'момент'), ('нормативный', 'акт'), ('настоящий', 'роль'), ('заметный', 'увеличение'), ('полный', 'смысл'), ('подобный', 'оценка'), ('существенный', 'влияние'), ('необходимый', 'условие'), ('целый', 'ряд'), ('технический', 'прогресс'), ('негативный', 'урок'), ('соответствующий', 'вывод'), ('значитель

In [18]:
# Считаем биграммы по всему тексту
tokens = nltk.word_tokenize(init_text)
bigrams = list(nltk.bigrams(tokens))

countered_bigrams = dict(collections.Counter(bigrams))

countered_bigrams

{('сущность', 'и'): 3,
 ('и', 'социальный'): 5,
 ('социальный', 'направленность'): 1,
 ('направленность', 'юридический'): 1,
 ('юридический', 'профессия'): 14,
 ('профессия', 'особенно'): 1,
 ('особенно', 'отчетливо'): 1,
 ('отчетливо', 'проявляться'): 1,
 ('проявляться', 'в'): 1,
 ('в', 'функция'): 1,
 ('функция', 'юрист'): 2,
 ('юрист', 'как'): 6,
 ('как', 'социально-профессиональный'): 1,
 ('социально-профессиональный', 'группа'): 1,
 ('группа', 'генеральный'): 1,
 ('генеральный', 'функция'): 1,
 ('функция', 'юридический'): 4,
 ('юридический', 'сообщество'): 2,
 ('сообщество', 'профессиональный'): 1,
 ('профессиональный', 'обеспечение'): 1,
 ('обеспечение', 'механизм'): 1,
 ('механизм', 'правовой'): 1,
 ('правовой', 'регулирование'): 5,
 ('регулирование', 'это'): 1,
 ('это', 'предполагать'): 1,
 ('предполагать', 'включение'): 1,
 ('включение', 'весь'): 1,
 ('весь', 'основной'): 2,
 ('основной', 'направление'): 2,
 ('направление', 'работа'): 1,
 ('работа', 'юрист'): 5,
 ('юрист', 'св

In [19]:
adjs_nouns = []
occur_freqs = []

for bigram, value in zip(countered_bigrams.keys(), countered_bigrams.values()):
    if bigram in adj_noun_bigrams:
        adjs_nouns.append(bigram)
        occur_freqs.append(value)

d = {'ПРИЛ.-СУЩ.':adjs_nouns, 'Частота встречаемости':occur_freqs}
print('РАНЖИРОВАНИЕ ПО МЕРЕ ЧАСТОТНОСТИ')
df = pd.DataFrame(data=d, index=None)
df = df.sort_values(by=['Частота встречаемости'], ascending=False) # по убыванию, поэтому ascending=False
df

РАНЖИРОВАНИЕ ПО МЕРЕ ЧАСТОТНОСТИ


,ПРИЛ.-СУЩ.,Частота встречаемости
4,"(российский, федерация)",72
35,"(внутренний, дело)",15
1,"(юридический, профессия)",14
55,"(местный, самоуправление)",10
26,"(действующий, законодательство)",9
42,"(служебный, обязанность)",8
87,"(учебный, заведение)",6
38,"(судебный, разбирательство)",5
77,"(квалификационный, экзамен)",5
2,"(правовой, регулирование)",5


### Мера взаимной информации

Взаимная информация — статистическая функция двух случайных величин, описывающая количество информации, содержащееся в одной случайной величине относительно другой.    
Для текстов эта мера количественно рассматривает случаи взаимного появления слов. 

In [20]:
N = len(words)
print('Размер корпуса N = ' + str(N) + ' слов.')


words_for_MI_analysis = []
for bigram in adjs_nouns:
    words_for_MI_analysis.append(bigram[0])
    words_for_MI_analysis.append(bigram[1])

words_freqs = collections.Counter(words)

words_MI, freqs_MI = [], []
for word, value in zip(words_freqs.keys(), words_freqs.values()):
    if word in words_for_MI_analysis:
        words_MI.append(word)
        freqs_MI.append(value)


# Для получения знаменателя формулы взаимной информации достаточно перемножить частоты каждого из элементов этого спсика
# (потому что слова в этом списке и так идут в правильном порядке ПРИЛ. - СУЩ.)
print(dict(zip(words_MI, freqs_MI)))

Размер корпуса N = 20695 слов.
{'социальный': 29, 'направленность': 7, 'юридический': 184, 'профессия': 44, 'группа': 13, 'профессиональный': 118, 'правовой': 99, 'регулирование': 7, 'основной': 21, 'направление': 8, 'российский': 109, 'федерация': 73, 'законотворческий': 1, 'процесс': 21, 'политический': 14, 'отношение': 44, 'противоречивый': 3, 'характер': 27, 'господствующий': 1, 'класс': 1, 'действующий': 15, 'законодательство': 37, 'неопределенный': 1, 'время': 57, 'физический': 20, 'лицо': 83, 'обозримый': 1, 'перспектива': 1, 'активный': 5, 'участие': 16, 'разнообразный': 2, 'форма': 17, 'первоначальный': 2, 'рабочий': 5, 'последний': 13, 'момент': 5, 'нормативный': 13, 'акт': 17, 'заметный': 4, 'увеличение': 1, 'полный': 11, 'смысл': 9, 'существенный': 8, 'влияние': 6, 'необходимый': 31, 'условие': 31, 'целый': 3, 'ряд': 14, 'технический': 1, 'прогресс': 1, 'негативный': 3, 'урок': 1, 'соответствующий': 21, 'вывод': 3, 'значительный': 6, 'затрата': 2, 'первостепенный': 1, 'знач

In [21]:
import numpy as np
# Считаем знаменатель взаимной информации
def calculate_MI_denominators():
    """Беру каждый нечётный элемент (= частоту первого слова словосочетания), кладу в первый список.
    Беру каждый чётный элемент списка (= частоту второго слова словосочетания), кладу во второй список.
    А потом элементы этих списков попарно перемножаю. 
    Получаю на выходе знаменатель для выражения взаимной информации для каждого слова."""
    
    denominators = []
    freqs_for_the_first_words = []
    freqs_for_the_second_words = []
    for i in range(0, len(freqs_MI)):
        if i % 2 == 0:
            freqs_for_the_first_words.append(freqs_MI[i])
        else:
            freqs_for_the_second_words.append(freqs_MI[i])
    for freq_pair in list(zip(freqs_for_the_first_words, freqs_for_the_second_words)):
        denominators.append(freq_pair[0] * freq_pair[1])
    return denominators


def calculate_MI(joint_occurrence, absolute_occurrence, N=N):    
    return np.log2((joint_occurrence / absolute_occurrence) * N)

MIs = list(map(calculate_MI, occur_freqs, calculate_MI_denominators()))


d = {'Прил. + Сущ.':adjs_nouns, 'Взаимная информация':np.array(MIs)}
print('РАНЖИРОВАНИЕ ПО МЕРЕ ВЗАИМНОЙ ИНФОРМАЦИИ')
df = pd.DataFrame(data=d, index=None)
df = df.sort_values(by=['Взаимная информация'], ascending=False) # по убыванию, поэтому ascending=False
df

РАНЖИРОВАНИЕ ПО МЕРЕ ВЗАИМНОЙ ИНФОРМАЦИИ


,Прил. + Сущ.,Взаимная информация
42,"(служебный, обязанность)",17.336995
35,"(внутренний, дело)",15.073960
87,"(учебный, заведение)",14.921957
77,"(квалификационный, экзамен)",14.658923
95,"(идейный, убежденность)",14.336995
119,"(горький, ирония)",14.336995
94,"(прагматический, настрой)",14.336995
116,"(андреевский, звезда)",14.336995
24,"(соответствующий, вывод)",14.336995
115,"(редакционный, обработка)",14.336995


### Что является термином среди словосочетаний с наибольшей частотой встречаемости в тексте

![caption](freqs_terms.png)

### Что является термином среди словосочетаний с наибольшим показателем взаимной информации

![caption](MI_terms_.png)

### Выводы

- Мера взаимной информации показывает, сколько количественно информации в одной величине относительно другой.
- Терминов больше при ранжировании по частоте